In [ ]:
%matplotlib inline

import numpy as np
from scipy import stats
from sklearn.gaussian_process import GaussianProcessClassifier
from matplotlib import pyplot as pl
import csv

In [ ]:
participant = 'stratus125'
# tested: 125, 132

In [ ]:
# The data is stored in a csv file that has three columns,
# one for each frequency and one for the percept report.

# Define a function that cleans this data (this is used for)
# both the test and the training data later:
def cleandata(data):
    assert data.shape[1] == 3, "Data not in the right format."
    
    # trim the data of all timepoints where nothing was reported
    # (this usually also removes the beginning and end of trials)
    data = data[np.logical_not(np.isnan(data[:, 2])), :]
    
    # downsample the data (to avoid memory issues!)
    downsamplefactor = 18  # choose how much you want to throw out
    data = data[np.arange(0, data.shape[0], downsamplefactor), :]
    
    # split into x and y
    x = data[:, 0:2]
    y = data[:, 2]
    return x, y

In [ ]:
# Load training data from the file matlab generates
traindata = np.genfromtxt(participant + '_sim.csv', delimiter=',', skip_header=1,
                          missing_values=['NaN', 'nan'], filling_values=None)

# Clean this data
trainx, trainy = cleandata(traindata)
traindata = None  # delete to save memory!

In [ ]:
# Create a gaussian process class
# nugget is the parameter that defines how much regularization there is
# gp = GaussianProcess(theta0=5e-1, thetaL=1e-3, thetaU=3,
#                      nugget=1e-12)
gp = GaussianProcessClassifier()

# train this class on the data
print('training...')
gp.fit(trainx, trainy)
print('fit done..!')

# Discard all training data to preserve memory
trainx = trainy = None

In [ ]:
# load test data
testdata = np.genfromtxt(participant + '_rival.csv', delimiter=',', skip_header=1,
                         missing_values=['NaN', 'nan'], filling_values=None)
testx, testy = cleandata(testdata)  # clean data
testdata = None  # clear from memory

In [ ]:
# now get a prediction for all points in the test data
print('predicting...')
predicty = gp.predict(testx)
proby = gp.predict_proba(testx)
print('predicted...!')

In [ ]:
# Categorical Predictions
colors = ['gray', 'red', 'blue']
num_bins = 50
for percept in [-1, 0, 1]:
    fig = pl.figure(percept)
    n, bins, patches = pl.hist(predicty[np.equal(testy, percept)], num_bins, facecolor=colors[percept], alpha=0.5)
    pl.title('Percept ' + str(percept))
    pl.xlim((-1.2, 1.2))


In [ ]:
# Prediction Probabilities
colors = ['gray', 'red', 'blue']
num_bins = 50
for percept in [-1, 0, 1]:
    fig = pl.figure(percept)
    n, bins, patches = pl.hist(proby[np.equal(testy, percept), 1], num_bins, facecolor=colors[percept], alpha=0.5)
    pl.title('Percept ' + str(percept))
    pl.xlim((-0.2, 1.2))